In [17]:
import sys
sys.path.append('../')

from baselines.FixTimeRegressor import FixTimeRegressor
from baselines.TimeRangeRegressor import TimeRangeRegressor

import pandas as pd
import tqdm
import numpy as np

import pyarrow.parquet as pq

In [18]:
regressor = TimeRangeRegressor(['05:00:00', '08:00:00'], ['21:00:00', '23:59:59'])

In [19]:
train_file = pq.ParquetFile('../data/raw/train_series.parquet')

In [20]:
participant_series_ids = ['038441c925bb', '03d92c9f6f8a', '0402a003dae9', '04f547b8017d', '05e1944c3818', '08db4255286f']
train_series = pd.DataFrame()
for series in participant_series_ids:
  train_series = pd.concat([train_series, pd.read_parquet("../data/raw/train_series.parquet", filters=[('series_id','=',series)])], axis=0)

In [22]:
results = regressor.predict(train_series)

results.head()

100%|██████████| 726105/726105 [00:45<00:00, 15908.18it/s]


,row_id,series_id,step,event,score
0,0,038441c925bb,11220,wakeup,0.5
1,1,038441c925bb,28200,wakeup,0.5
2,2,038441c925bb,44400,wakeup,0.5
3,3,038441c925bb,45360,wakeup,0.5
4,4,038441c925bb,45900,wakeup,0.5


In [23]:
results.to_csv('../results/baseline_submission_1.csv')

In [24]:
regressor = FixTimeRegressor('06:00:00', '22:00:00')

In [25]:
results = regressor.predict(train_series)

100%|██████████| 346/346 [00:00<00:00, 13870.11it/s]


In [26]:
results.to_csv('../results/baseline_submission.csv', sep=',', index=False)